Updating the SED files for MEATS to newer MAST standards

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
from astropy.convolution import convolve, Box1DKernel
from scipy.interpolate import interp1d

import astropy.units as u
from astropy import time, coordinates as coord
import astropy.constants as const
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord
from matplotlib.gridspec import GridSpec
from datetime import datetime


#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

In [43]:
oldv = 1
version = 25


    STIS spectra

DOI = ‘10.17909/T9DG6F’

EQUINOX =’2000.2’ or RADESYS = 'ICRS'

HLSPID =’MUSCLES’

HLSPVER=’V1’

LICENSE =’CC BY 4.0’

LICENSURL= 'https://creativecommons.org/licenses/by/4.0/'

TIMESYS: Character string code (see FITS Standard) for the time scale of time-related keywords. Ex: 'UTC'

 

    Model spectra

The same set of metadata is required as STIS data except for EQUINOX/RADESYS and TIMESYS.

 

    SED data 

The same set of metadata is required as STIS data except for OBSERVAT.

MJD-BEG and MJD-END in the primary header. (* see below)

 

    XMM data

The same set of metadata is required as STIS data.

RA_TARG and DEC_TARG values are missing. Please include the correct RA/Dec values.

In [22]:
stars = os.listdir('draft_hlsp/')
stars

['hd120411',
 'toi-776_test',
 'k2-18',
 'kepler-51',
 'wasp-63',
 'toi-776',
 'kap1cet',
 'gj_341',
 'tau_ceti',
 'toi-421',
 'gj_367',
 'wasp-121']

['draft_hlsp/hd120411/hlsp_muscles_hst_stis_hd-120411_g230l_v1_component-spec.fits'
 'draft_hlsp/hd120411/hlsp_muscles_hst_stis_hd-120411_g140l_v1_component-spec.fits'
 'draft_hlsp/hd120411/hlsp_muscles_hst_stis_hd-120411_g430l_v1_component-spec.fits']
['draft_hlsp/hd120411/hlsp_muscles_model_dem_hd-120411_na_v1_component-spec.fits', 'draft_hlsp/hd120411/hlsp_muscles_model_apec_hd-120411_na_v1_component-spec.fits', 'draft_hlsp/hd120411/hlsp_muscles_model_phx_hd-120411_na_v1_component-spec.fits', 'draft_hlsp/hd120411/hlsp_muscles_model_lya_hd-120411_na_v1_component-spec.fits']
['draft_hlsp/hd120411/hlsp_muscles_multi_multi_hd-120411_broadband_v1_adapt-const-res-sed.fits', 'draft_hlsp/hd120411/hlsp_muscles_multi_multi_hd-120411_broadband_v1_var-res-sed.fits', 'draft_hlsp/hd120411/hlsp_muscles_multi_multi_hd-120411_broadband_v1_adapt-var-res-sed.fits', 'draft_hlsp/hd120411/hlsp_muscles_multi_multi_hd-120411_broadband_v1_const-res-sed.fits']
['draft_hlsp/hd120411/hlsp_muscles_cxo_acis_hd-1

In [48]:

newpath = 'fixed_hlsp/'
for star in stars:
    newdir = '{}{}'.format(newpath, star)
    if not os.path.exists(newdir):
       os.makedirs(newdir)
    
    oldv = 1
    if star == 'wasp-63':
        oldv = 2
    starpath = 'draft_hlsp/{}/'.format(star)
    specs = np.hstack((glob.glob('{}*stis*.fits'.format(starpath)),glob.glob('{}*cos*.fits'.format(starpath))))
    mods = glob.glob('{}*mod*.fits'.format(starpath))
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*.fits'.format(starpath))))

    # print(specs)
    # print(mods)
    # print(seds)
    # print(xrays)
    
    
    starts = []
    ends = []

    for spec in specs:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        ra, dec = hdr['RA_TARG'], hdr['DEC_TARG']
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in mods:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in xrays:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('RA_TARG', ra))
        hdr.append(('DEC_TARG', dec))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    start = np.min(starts)
    end = np.max(ends)
    
    for spec in seds:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('MJD-BEG', start))
        hdr.append(('MJD-END', end))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)


In [40]:
hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'MOD     '                                                            
INSTRU00= 'LYA     '                                                            
GRATIN00= 'YOUNG   '                                                            
TELESC01= 'HST     '                                                            
INSTRU01= 'STS     '                                                            
GRATIN01= 'G140L   '        